# Notebook that handles fine-tuning from start to finish

In [3]:
!pip install transformers -U

  Using cached transformers-4.31.0-py3-none-any.whl (7.4 MB)
  Using cached huggingface_hub-0.16.4-py3-none-any.whl (268 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 771.9/771.9 kB 7.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 39.5 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.5 MB/s eta 0:00:00:00:01

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [4]:
!pip install accelerate==0.20.3

  Using cached accelerate-0.20.3-py3-none-any.whl (227 kB)
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.19.0
    Uninstalling accelerate-0.19.0:
      Successfully uninstalled accelerate-0.19.0

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [41]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 5.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 3.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 14.1 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 2.3 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 5.4 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.7/225.7 kB 3.9 MB/s eta 0:00:0000:01

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [67]:
import numpy as np
from sagemaker import get_execution_role
import boto3
import pandas as pd
from io import StringIO # Python 3.
from datasets import load_dataset,Dataset,DatasetDict

from transformers import DataCollatorWithPadding,AutoModelForSequenceClassification, Trainer, TrainingArguments,AutoTokenizer,AutoModel,AutoConfig
from transformers.modeling_outputs import TokenClassifierOutput
import torch
import torch.nn as nn
import pandas as pd
import json

from models.EDdisposition import EDdispositionClassifier

In [16]:
# reads out files

import boto3

s3 = boto3.resource('s3')
bucket = s3.Bucket('chianglab-dataderivatives')

folders = set()

for obj in bucket.objects.all():
    prefix, delimiter, _ = obj.key.rpartition('/')
    if prefix:
        folders.add(prefix + '/')

print('Folders:')
for folder in folders:
    print(folder)
    


Folders:
mimic-iv-clinical-database-demo-2.2/
mimic-iv-ed-2.2/
mimic-iv-2.2/
mimic-iv-ed-demo-2.2/


In [18]:
bucket = 'chianglab-dataderivatives'
subfolder = 'mimic-iv-ed-demo-2.2/'
conn = boto3.client('s3')
contents = conn.list_objects(Bucket=bucket, Prefix=subfolder)['Contents']

file_list = []

for f in contents:
    print(f['Key'])
    file_list.append(f['Key'][36:])

print(file_list)

mimic-iv-ed-demo-2.2/
mimic-iv-ed-demo-2.2/text_repr.json
['', '']


### Load in Data and reformat 

In [33]:
s3 = boto3.resource('s3')
bucket_name = 'chianglab-dataderivatives'
file_path = "mimic-iv-clinical-database-demo-2.2/admissions_text.json"


content_object = s3.Object(bucket_name, file_path)
file_content = content_object.get()['Body'].read().decode('utf-8')
json_content = json.loads(file_content)

df = pd.DataFrame(list(json_content.items()), columns = ['ID', 'ad_headline'])

In [34]:
# add codes 

bucket_name = 'chianglab-dataderivatives'
file_path = "mimic-iv-clinical-database-demo-2.2/codes_text.json"


content_object = s3.Object(bucket_name, file_path)
file_content = content_object.get()['Body'].read().decode('utf-8')
json_content = json.loads(file_content)
df['codes_headline'] = df['ID'].map(json_content)

# add meds
bucket_name = 'chianglab-dataderivatives'
file_path = "mimic-iv-clinical-database-demo-2.2/meds_text.json"

content_object = s3.Object(bucket_name, file_path)
file_content = content_object.get()['Body'].read().decode('utf-8')
json_content = json.loads(file_content)
df['meds_headline'] = df['ID'].map(json_content)

# add labs
bucket_name = 'chianglab-dataderivatives'
file_path = "mimic-iv-clinical-database-demo-2.2/labs_text.json"

content_object = s3.Object(bucket_name, file_path)
file_content = content_object.get()['Body'].read().decode('utf-8')
json_content = json.loads(file_content)
# need to change float IDs to int but then back to a string to map
res = dict()
for key in json_content:
    res[str(int(float(key)))] = json_content[key]
    
df['labs_headline'] = df['ID'].map(res)

### Data we will be working with

In [96]:
bucket_name = 'chianglab-dataderivatives'
file_path = "mimic-iv-ed-demo-2.2/text_repr.json"


content_object = s3.Object(bucket_name, file_path)
file_content = content_object.get()['Body'].read().decode('utf-8')
json_content = json.loads(file_content)
df = pd.DataFrame(json_content).T
print("length of dataframe: "+ str(len(df)))
df.head(5)
# df['codes_headline'] = df['ID'].map(json_content)


length of dataframe: 210


,arrival,eddischarge,admission,discharge,triage,medrecon,vitals,pyxis,codes
37887480,"Patient 10014729, a 21 year old white - other ...",The ED disposition was admitted at 2125-03-19 ...,The patient was admitted at 2125-03-19 16:58:00.,The patient's discharge disposition was: home ...,"At triage: temperature was 99.1, pulse was 90....",The patient was previously taking the followin...,The patient had the following vitals: At 2125-...,The patient received the following medications...,The patient received the following diagnostic ...
34176810,"Patient 10018328, a 83 year old white female, ...",The ED disposition was admitted at 2154-02-05 ...,The patient was admitted at 2154-02-05 21:58:00.,The patient's discharge disposition was: home ...,"At triage: temperature was 97.7, pulse was 74....",The patient was previously taking the followin...,The patient had the following vitals: At 2154-...,NaN,The patient received the following diagnostic ...
32103106,"Patient 10018328, a 83 year old white female, ...",The ED disposition was home at 2154-08-03 22:2...,The patient was not admitted.,The patient was not admitted.,"At triage: temperature was 96.2, pulse was 74....",The patient was previously taking the followin...,The patient had the following vitals: At 2154-...,The patient received the following medications...,The patient received the following diagnostic ...
38797992,"Patient 10020640, a 91 year old white female, ...",The ED disposition was admitted at 2153-02-13 ...,The patient was admitted at 2153-02-13 00:22:00.,The patient's discharge disposition was: skill...,"At triage: temperature was 99.2, pulse was 130...",The patient was previously taking the followin...,The patient had the following vitals: At 2153-...,The patient received the following medications...,The patient received the following diagnostic ...
33473053,"Patient 10015272, a 78 year old white female, ...",The ED disposition was admitted at 2137-06-12 ...,The patient was admitted at 2137-06-12 18:36:00.,The patient's discharge disposition was: home ...,"At triage: temperature was 97.5, pulse was 118...",The patient was previously taking the followin...,The patient had the following vitals: At 2137-...,The patient received the following medications...,The patient received the following diagnostic ...


In [97]:
df['eddischarge'] = [1 if 'admitted' in s.lower() else 0 for s in df['eddischarge']] # admitted = 1, Home = 0
df = df[[col for col in df.columns if col != 'eddischarge'] + ['eddischarge']] # rearrange column to the end
df = df.fillna("[PAD]")
df

,arrival,admission,discharge,triage,medrecon,vitals,pyxis,codes,eddischarge
37887480,"Patient 10014729, a 21 year old white - other ...",The patient was admitted at 2125-03-19 16:58:00.,The patient's discharge disposition was: home ...,"At triage: temperature was 99.1, pulse was 90....",The patient was previously taking the followin...,The patient had the following vitals: At 2125-...,The patient received the following medications...,The patient received the following diagnostic ...,1
34176810,"Patient 10018328, a 83 year old white female, ...",The patient was admitted at 2154-02-05 21:58:00.,The patient's discharge disposition was: home ...,"At triage: temperature was 97.7, pulse was 74....",The patient was previously taking the followin...,The patient had the following vitals: At 2154-...,[PAD],The patient received the following diagnostic ...,1
32103106,"Patient 10018328, a 83 year old white female, ...",The patient was not admitted.,The patient was not admitted.,"At triage: temperature was 96.2, pulse was 74....",The patient was previously taking the followin...,The patient had the following vitals: At 2154-...,The patient received the following medications...,The patient received the following diagnostic ...,0
38797992,"Patient 10020640, a 91 year old white female, ...",The patient was admitted at 2153-02-13 00:22:00.,The patient's discharge disposition was: skill...,"At triage: temperature was 99.2, pulse was 130...",The patient was previously taking the followin...,The patient had the following vitals: At 2153-...,The patient received the following medications...,The patient received the following diagnostic ...,1
33473053,"Patient 10015272, a 78 year old white female, ...",The patient was admitted at 2137-06-12 18:36:00.,The patient's discharge disposition was: home ...,"At triage: temperature was 97.5, pulse was 118...",The patient was previously taking the followin...,The patient had the following vitals: At 2137-...,The patient received the following medications...,The patient received the following diagnostic ...,1
...,...,...,...,...,...,...,...,...,...
30272878,"Patient 10038999, a 45 year old white male, ar...",The patient was admitted at 2131-05-22 21:49:00.,The patient's discharge disposition was: home ...,"At triage: temperature was not recorded, pulse...",[PAD],The patient had the following vitals: At 2131-...,The patient received the following medications...,The patient received the following diagnostic ...,1
31628990,"Patient 10009049, a 56 year old white male, ar...",The patient was admitted at 2174-05-26 08:21:00.,The patient's discharge disposition was: home ...,"At triage: temperature was 99.0, pulse was 87....",The patient was previously taking the followin...,The patient had the following vitals: At 2174-...,The patient received the following medications...,The patient received the following diagnostic ...,1
32405286,"Patient 10004457, a 65 year old white male, ar...",The patient was admitted at 2141-08-12 16:02:00.,The patient's discharge disposition was: home ...,"At triage: temperature was 97.6, pulse was 103...",The patient was previously taking the followin...,The patient had the following vitals: At 2141-...,The patient received the following medications...,The patient received the following diagnostic ...,1
34391979,"Patient 10004720, a 61 year old white male, ar...",The patient was admitted at 2186-11-12 18:01:00.,The patient's discharge disposition was: died ...,"At triage: temperature was not recorded, pulse...",The patient was previously taking the followin...,The patient had the following vitals: At 2186-...,The patient received the following medications...,The patient received the following diagnostic ...,1


In [98]:
col_names = df.columns.drop("eddischarge")
l = []
for i in col_names:
    temp = df[[i, 'eddischarge']].reset_index()
    temp = temp.sort_values(by=['index']).reset_index() # we sort the patient ID numerically before dropping it to preserve order in encoding
    temp = temp.drop(columns=["index", "level_0"])
    l.append(temp)
    print("\""+i+ "\" Dataframe")
    display(temp)

"arrival" Dataframe


,arrival,eddischarge
0,"Patient 10014354, a 60 year old white male, ar...",1
1,"Patient 10010867, a 28 year old white - brazil...",1
2,"Patient 10002930, a 48 year old black/african ...",1
3,"Patient 10005866, a 57 year old portuguese mal...",1
4,"Patient 10038999, a 45 year old white male, ar...",1
...,...,...
205,"Patient 10023117, a 53 year old white male, ar...",1
206,"Patient 10002930, a 48 year old black/african ...",0
207,"Patient 10020740, a 56 year old white male, ar...",0
208,"Patient 10015860, a 53 year old white male, ar...",0


"admission" Dataframe


,admission,eddischarge
0,The patient was admitted at 2147-11-14 22:12:00.,1
1,The patient was admitted at 2147-12-30 08:40:00.,1
2,The patient was admitted at 2198-04-17 19:38:00.,1
3,The patient was admitted at 2149-09-20 14:30:00.,1
4,The patient was admitted at 2131-05-22 21:49:00.,1
...,...,...
205,The patient was admitted at 2174-06-07 23:25:00.,1
206,The patient was admitted at 2201-02-12 16:58:00.,0
207,The patient was admitted at 2150-09-15 14:09:00.,0
208,The patient was admitted at 2191-01-15 01:55:00.,0


"discharge" Dataframe


...

,vitals,eddischarge
0,The patient had the following vitals: At 2147-...,1
1,The patient had the following vitals: At 2147-...,1
2,The patient had the following vitals: At 2198-...,1
3,The patient had the following vitals: At 2149-...,1
4,The patient had the following vitals: At 2131-...,1
...,...,...
205,The patient had the following vitals: At 2174-...,1
206,The patient had the following vitals: At 2201-...,0
207,The patient had the following vitals: At 2150-...,0
208,The patient had the following vitals: At 2191-...,0


"pyxis" Dataframe


,pyxis,eddischarge
0,The patient received the following medications...,1
1,The patient received the following medications...,1
2,The patient received the following medications...,1
3,The patient received the following medications...,1
4,The patient received the following medications...,1
...,...,...
205,[PAD],1
206,The patient received the following medications...,0
207,The patient received the following medications...,0
208,The patient received the following medications...,0


"codes" Dataframe


,codes,eddischarge
0,The patient received the following diagnostic ...,1
1,The patient received the following diagnostic ...,1
2,The patient received the following diagnostic ...,1
3,The patient received the following diagnostic ...,1
4,The patient received the following diagnostic ...,1
...,...,...
205,The patient received the following diagnostic ...,1
206,The patient received the following diagnostic ...,0
207,The patient received the following diagnostic ...,0
208,The patient received the following diagnostic ...,0


In [99]:
# Indices for sub-dataframes, that will each individually be tokenized
# 0 - arrival
# 1 - admission
# 2 - discharge
# 3 - triage
# 4 - medrecon
# 5 - vitals
# 6 - pyxis
# 7 - codes
arrival_hf=Dataset.from_pandas(l[0])
admission_hf=Dataset.from_pandas(l[1])
discharge_hf=Dataset.from_pandas(l[2])
triage_hf=Dataset.from_pandas(l[3])
medrecon_hf=Dataset.from_pandas(l[4])
vitals_hf=Dataset.from_pandas(l[5])
pyxis_hf=Dataset.from_pandas(l[6])
codes_hf=Dataset.from_pandas(l[7])

# Sanity check printer
printer = [arrival_hf, admission_hf, discharge_hf, triage_hf, medrecon_hf, vitals_hf, pyxis_hf, codes_hf]
for i in printer:
    print(i)

Dataset({
    features: ['arrival', 'eddischarge'],
    num_rows: 210
})
Dataset({
    features: ['admission', 'eddischarge'],
    num_rows: 210
})
Dataset({
    features: ['discharge', 'eddischarge'],
    num_rows: 210
})
Dataset({
    features: ['triage', 'eddischarge'],
    num_rows: 210
})
Dataset({
    features: ['medrecon', 'eddischarge'],
    num_rows: 210
})
Dataset({
    features: ['vitals', 'eddischarge'],
    num_rows: 210
})
Dataset({
    features: ['pyxis', 'eddischarge'],
    num_rows: 210
})
Dataset({
    features: ['codes', 'eddischarge'],
    num_rows: 210
})


In [100]:
# grabs a tokenizer from the distilbert-base-uncased model
model = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model)
tokenizer.model_max_len=512

In [102]:
def tokenize_arrival(batch):
  return tokenizer(batch["arrival"], truncation=True, max_length=512)
def tokenize_admission(batch):
  return tokenizer(batch["admission"], truncation=True, max_length=512)
def tokenize_discharge(batch):
  return tokenizer(batch["discharge"], truncation=True, max_length=512)
def tokenize_triage(batch):
  return tokenizer(batch["triage"], truncation=True, max_length=512)
def tokenize_medrecon(batch):
  return tokenizer(batch["medrecon"], truncation=True, max_length=512)
def tokenize_vitals(batch):
  return tokenizer(batch["vitals"], truncation=True, max_length=512)
def tokenize_pyxis(batch):
  return tokenizer(batch["pyxis"], truncation=True, max_length=512)
def tokenize_codes(batch):
  return tokenizer(batch["codes"], truncation=True, max_length=512)


tokenized_arrival = arrival_hf.map(tokenize_arrival, batched=True)
tokenized_admission = admission_hf.map(tokenize_admission, batched=True)
tokenized_discharge = discharge_hf.map(tokenize_discharge, batched=True)
tokenized_triage = triage_hf.map(tokenize_triage, batched=True)
tokenized_medrecon = medrecon_hf.map(tokenize_medrecon, batched=True)
tokenized_vitals = vitals_hf.map(tokenize_vitals, batched=True)
tokenized_pyxis = pyxis_hf.map(tokenize_pyxis, batched=True)
tokenized_codes = codes_hf.map(tokenize_codes, batched=True)

# Sanity check printer
printer = [tokenized_arrival, tokenized_admission, tokenized_discharge, tokenized_triage, tokenized_medrecon, tokenized_vitals, tokenized_pyxis, tokenized_codes]
for i in printer:
    print(i)

Map: 100%|██████████| 210/210 [00:00<00:00, 5187.97 examples/s]

Dataset({
    features: ['arrival', 'eddischarge', 'input_ids', 'attention_mask'],
    num_rows: 210
})
Dataset({
    features: ['admission', 'eddischarge', 'input_ids', 'attention_mask'],
    num_rows: 210
})
Dataset({
    features: ['discharge', 'eddischarge', 'input_ids', 'attention_mask'],
    num_rows: 210
})
Dataset({
    features: ['triage', 'eddischarge', 'input_ids', 'attention_mask'],
    num_rows: 210
})
Dataset({
    features: ['medrecon', 'eddischarge', 'input_ids', 'attention_mask'],
    num_rows: 210
})
Dataset({
    features: ['vitals', 'eddischarge', 'input_ids', 'attention_mask'],
    num_rows: 210
})
Dataset({
    features: ['pyxis', 'eddischarge', 'input_ids', 'attention_mask'],
    num_rows: 210
})
Dataset({
    features: ['codes', 'eddischarge', 'input_ids', 'attention_mask'],
    num_rows: 210
})
